In [ ]:
# Instalando biblioteca para o usar o github

In [ ]:
# !pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
from pathlib import Path
import shutil

dirpath = Path('./') / 'datasets'
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)

In [2]:
!git clone https://github.com/naubergois/datasets.git --depth 1

Cloning into 'datasets'...
remote: Enumerating objects: 4078, done.
remote: Counting objects: 100% (1115/1115), done.
remote: Compressing objects: 100% (773/773), done.
error: 7507 bytes of body are still expectedMiB | 404.00 KiB/s 
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [ ]:
! cp ./datasets/ForSpark/confs/* ./

In [ ]:
import sys
from pyspark.sql import *
from datasets.ForSpark.lib.logger import Log4j
from datasets.ForSpark.lib.utils import *

In [ ]:
conf = get_spark_app_config()

spark = SparkSession \
    .builder \
    .appName("HelloSpark") \
    .master("local[2]") \
    .getOrCreate()

logger = Log4j(spark)

In [ ]:
logger.info("Starting HelloSpark")

In [ ]:
survey_raw_df=spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv('./datasets/ForSpark/sample.csv')


In [ ]:
survey_raw_df.show()

+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+----------+-----------------------+---------------------+------------------+---------------+--------+
|          Timestamp|Age|Gender|       Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|   coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+--

In [ ]:
partitioned_survey_df = survey_raw_df.repartition(2)


In [ ]:
count_df =partitioned_survey_df.filter("Age < 40") \
        .select("Age", "Gender", "Country", "state") \
        .groupBy("Country") \
        .count()

In [ ]:
count_df.show()

logger.info("Finished HelloSpark")
spark.stop()

+--------------+-----+
|       Country|count|
+--------------+-----+
| United States|    4|
|        Canada|    2|
|United Kingdom|    1|
+--------------+-----+

